In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os 
from tqdm import tqdm 

In [25]:
def binarize_traces(df, threshold):
    """
    Return a copy of df where every value < threshold becomes 0,
    and every value >= threshold becomes 1.
    """
    return (df >= threshold).astype(int)

def get_event_rate(df, samplingRate):
    """
    Given a binarized DataFrame `df` of shape (n_frames, n_cells),
    compute, for each cell (column), the number of “events” in each
    forward-looking 1-second window. Returns a DataFrame of shape
    (n_frames - samplingRate + 1, n_cells).
    """
    # 1) grab the raw values (frames × cells)
    arr = df.values.astype(int)  
    # 2) build a “boxcar” kernel of length = samples per second
    kernel = np.ones(samplingRate, dtype=int)
    # 3) convolve along the time-axis for each cell (axis=0)
    #    mode='valid' gives you only positions where the full window fits
    rates = np.apply_along_axis(
        lambda col: np.convolve(col, kernel, mode='valid'),
        axis=0,
        arr=arr
    )
    # 4) build a new index so that row i corresponds to window df.index[i : i+samplingRate]
    #    if your original df.index is RangeIndex starting at 0, this is simply 0..n-sR
    new_index = df.index[: rates.shape[0]]
    return pd.DataFrame(rates, index=new_index, columns=df.columns)

def get_indices_by_speed_bins(df, speed_col, bins):
    """
    For each (low, high) in `bins`, find the row-indices where
    low <= df[speed_col] < high (for high == inf, just low <=).
    
    Returns
    -------
    dict
        keys   : tuple (low, high)
        values : list of row-index labels in df
    """
    bin_indices = {}
    for low, high in bins:
        if np.isinf(high):
            mask = df[speed_col] >= low
        else:
            mask = (df[speed_col] >= low) & (df[speed_col] < high)
        bin_indices[(low, high)] = df.index[mask].tolist()
    return bin_indices

def avg_event_rate_by_speed_bins(df, idx_by_bin, speed_col='linearSpeedcmPerSecond'):
    """
    Given:
      • df            : DataFrame with one column per cell (instantaneous rates)
                        plus a `speed_col`.
      • idx_by_bin    : dict mapping (low, high) tuples → list of row-indices
                        where speed ∈ [low, high) (or >= low if high is inf).
      • speed_col     : name of the speed column in df (will be dropped).
    Returns:
      • avg_df        : DataFrame indexed by your (low, high) bins,
                        columns are the cell-names, entries are the mean
                        event-rate of that cell over all frames in that bin.
    """
    # 1) Identify all “cell” columns (everything except the speed column)
    event_cols = [c for c in df.columns if c != speed_col]

    # 2) Make a DataFrame to hold means; use a MultiIndex of your bin tuples
    bin_index = pd.MultiIndex.from_tuples(idx_by_bin.keys(), names=['low','high'])
    avg_df = pd.DataFrame(index=bin_index, columns=event_cols, dtype=float)

    # 3) For each bin, pull those rows and take the column‐wise mean
    for bin_range, idxs in idx_by_bin.items():
        if len(idxs)>0:
            avg_df.loc[bin_range] = df.loc[idxs, event_cols].mean()
        else:
            avg_df.loc[bin_range] = np.nan

    return avg_df


def apply_spatial_filter(
    df,
    frame_rate: int,
    square_box_threshold: float,
    coord_cols=('X_coor','Y_coor'),
    vel_col='velocity_PixelsPerSec'
):
    """
    Slide a window of `frame_rate` frames over the trajectory;
    whenever the animal stays within a ±square_box_threshold pixel
    box (around the start‐of‐window position) for the whole window,
    zero out the velocity over that window.

    Returns a new numpy array of filtered velocities.
    """
    X = df[coord_cols[0]].to_numpy()
    Y = df[coord_cols[1]].to_numpy()
    vel = df[vel_col].to_numpy().copy()
    n = len(df)
    w = frame_rate

    for i in range(n):
        end = min(i + w, n)
        cx, cy = X[i], Y[i]
        Xw, Yw = X[i:end], Y[i:end]

        # check if all points stay in the box
        if ((Xw >= cx - square_box_threshold) & (Xw <= cx + square_box_threshold)).all() \
        and ((Yw >= cy - square_box_threshold) & (Yw <= cy + square_box_threshold)).all():
            vel[i:end] = 0

    return vel

In [36]:

#path to eZTrack data
dirPath = '/Users/johnmarshall/Documents/Analysis/miniscope_analysis/miniscopeLinearTrack/2025.1/m989/2025_01_04/'

alignedFile = '1_24_mc_989_1_4_25_Run12cellTracesAlignedToTracking.csv'

#load calcium signal aligned to tracking data 
alignedTraces = pd.read_csv(dirPath+alignedFile)
#separate out location data 
alignedTracesLocationData = alignedTraces[['closestBehavCamFrameIdx', 'X_coor', 'Y_coor']]
#separate out cell traces
aligned_cell_traces = alignedTraces.loc[:, alignedTraces.columns.str.startswith('cell_')]
#threshold signal 
firingThresholdSD = 2.5
signalPeaks = binarize_traces(aligned_cell_traces, firingThresholdSD)
samplingRate = 20 
instantaneousEventRate = get_event_rate(signalPeaks, samplingRate)

#this calculates the velocity in pixels per second from both the x and y coordinates 
alignedTraces['velocity_PixelsPerSec'] = (np.hypot(
    alignedTraces['X_coor'].diff(),
    alignedTraces['Y_coor'].diff()
    ) * samplingRate
).fillna(0)

#take just the X coordinate column and calculate a X velocity (in pixels)
samplingRate = 20 
linearSpeed = abs(alignedTraces['X_coor'].diff() * samplingRate)
linearSpeed.fillna(0, inplace=True)
pixelsPercm = 3.8
linearSpeedcmPerSecond = linearSpeed / pixelsPercm

#append to event rate dataframe 
instantaneousEventRate['linearSpeedcmPerSecond'] = linearSpeedcmPerSecond

#perform "spatial filter" on 'velocity_PixelsPerSec' trace 
frameRate = 5
squareBoxThreshold = 10

alignedTraces['velocitySpatialFiltered'] = apply_spatial_filter(
    alignedTraces,
    frame_rate=frameRate,
    square_box_threshold=squareBoxThreshold,
    coord_cols=('X_coor','Y_coor'),
    vel_col='velocity_PixelsPerSec'   # change this if your velocity column is named differently
)

#perform temporal filter on speed trace
window = int(samplingRate/5)  
alignedTraces['velocitySmooth'] = (alignedTraces['velocitySpatialFiltered'].rolling(window, center=True, min_periods=1).mean())

#convert to cm 
velocity2dPer_cm_Filtered = alignedTraces['velocitySpatialFiltered'] / pixelsPercm

#append to event rate dataframe 
alignedTraces['velocity2dSpatialFiltered'] = velocity2dPer_cm_Filtered
instantaneousEventRate['velocity2dSpatialFiltered'] = velocity2dPer_cm_Filtered

#calculate, for each cell, event rate at different speed bins 
bins = [(0, 0.5), (0.5, 5), (5, 10), (10, 15), (15, np.inf)]

idx_by_bin = get_indices_by_speed_bins(instantaneousEventRate, 'velocity2dSpatialFiltered', bins)
#avg rates by speed 
avg_rates = avg_event_rate_by_speed_bins(instantaneousEventRate, idx_by_bin, speed_col='velocity2dSpatialFiltered')

In [37]:
avg_rates.mean(axis=1)

low   high
0.0   0.5     0.555232
0.5   5.0     0.406810
5.0   10.0    0.639256
10.0  15.0    0.522743
15.0  inf     1.055147
dtype: float64

In [38]:
# 1) compute the per-frame average of every numeric column
binned = (
    alignedTraces
      .groupby('closestBehavCamFrameIdx')           # group by frame-idx
      .mean(numeric_only=True)                     # mean of only number-dtype cols
)

In [39]:
binned.to_csv(dirPath+'/velocityBehavIdx.csv')
binned

,cell_2,cell_3,cell_4,cell_6,cell_7,cell_9,cell_10,cell_11,cell_12,cell_13,...,Frame Number,Time Stamp (ms),Buffer Index,X_coor,Y_coor,Distance_px,velocity_PixelsPerSec,velocitySpatialFiltered,velocitySmooth,velocity2dSpatialFiltered
closestBehavCamFrameIdx,,,,,,,,,,,,,,,,,,,,,
0,1.668685,0.294092,0.428221,-0.351496,-0.831072,0.109596,-0.040073,-0.634220,0.631596,0.602733,...,0.5,-21.0,0.0,19.688645,30.297362,0.000000,0.000000,0.0,0.0,0.0
1,1.224337,-0.085234,0.200603,-0.749567,-0.437487,-0.251298,0.125360,-0.860186,0.245964,0.114591,...,2.0,53.0,0.0,19.543148,30.519642,0.265665,5.313302,0.0,0.0,0.0
2,0.765238,-0.127494,-0.018520,-0.709715,-0.431082,-0.076383,-0.011789,-0.993659,0.185247,0.059971,...,3.0,104.0,0.0,19.709833,30.326511,0.255115,5.102293,0.0,0.0,0.0
3,1.072975,-0.258168,0.269059,-1.173081,-0.558063,0.041817,0.048384,-0.607870,0.764832,0.243144,...,4.5,179.5,0.0,19.761570,30.151749,0.182260,1.822596,0.0,0.0,0.0
4,1.017623,-0.607175,0.301940,-1.202772,-0.902351,0.074842,0.201642,-0.601713,0.715616,-0.596461,...,6.0,256.0,0.0,19.820763,29.684470,0.471013,9.420267,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17200,0.011064,-1.562486,-1.166543,-0.822912,2.550410,-2.029023,-1.076586,1.569866,-1.190951,0.546112,...,23685.0,1199791.0,0.0,19.284412,31.285721,0.252116,5.042315,0.0,0.0,0.0
17201,-0.151820,-1.420034,-1.224491,-0.726929,2.658755,-1.504129,-1.184012,0.816136,-1.515178,0.286633,...,23686.0,1199841.0,0.0,19.048073,31.483692,0.308300,6.165993,0.0,0.0,0.0
17202,-0.087203,-0.893916,-1.403545,-0.437668,2.241924,-2.043473,-1.253649,1.331484,-1.446298,0.316783,...,23687.5,1199917.0,0.0,18.953934,31.496193,0.094965,0.949653,0.0,0.0,0.0


In [40]:
##validate with video overlay  
#small video 

# ── CONFIG ─────────────────────────────────────────────────────────────────────

dirPath      = '/Users/johnmarshall/Documents/Analysis/miniscope_analysis/miniscopeLinearTrack/2025.1/m989/2025_01_04/My_WebCam/'
rotatedVideo = 'video_output.avi'
input_vid    = os.path.join(dirPath, rotatedVideo)
output_vid   = os.path.join(dirPath, 'with_velocity_overlay.mp4')

# your velocity array (length == # frames)
vel_array = binned['velocity2dSpatialFiltered'].to_numpy()

# your per-behavior-frame X positions (make sure it's indexed by frame 0…n-1)
# e.g. binned = binned.reset_index().set_index('frame')
x_pos = binned['X_coor'].to_numpy()

# height of the black bar you want to add
bar_h = 50

# ── SET UP I/O ─────────────────────────────────────────────────────────────────

cap      = cv2.VideoCapture(input_vid)
fps      = cap.get(cv2.CAP_PROP_FPS)
w        = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h        = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# choose a codec that FIJI likes for AVIs
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out    = cv2.VideoWriter(output_vid, fourcc, fps, (w, h + bar_h))

# text & dot styling
font        = cv2.FONT_HERSHEY_SIMPLEX
font_scale  = 1
thickness   = 2
margin      = 10
dot_radius  = 5

# ── PROCESS FRAMES ──────────────────────────────────────────────────────────────

for frame_idx in tqdm(range(n_frames-1), desc="Adding overlay"):
    ret, frame = cap.read()
    if not ret:
        break

    # 1) create the padded frame
    bar       = np.zeros((bar_h, w, 3), dtype=np.uint8)
    new_frame = np.vstack((frame, bar))

    # 2) overlay the velocity text in the bar (left‐aligned)
    vel_text = f"{vel_array[frame_idx-1]:.2f} cm/s"
    (tw, th), _ = cv2.getTextSize(vel_text, font, font_scale, thickness)
    text_x = margin
    text_y = h + margin + th
    cv2.putText(
        new_frame,
        vel_text,
        (text_x, text_y),
        font,
        font_scale,
        (255, 255, 255),
        thickness,
        cv2.LINE_AA
    )

    # 3) draw the dot at the mouse’s X in the center of the bar
    xm = int(np.clip(x_pos[frame_idx-1], 0, w-1))
    y_dot = h + bar_h // 2
    cv2.circle(new_frame, (xm, y_dot), dot_radius, (255, 255, 255), -1)

    out.write(new_frame)

# ── CLEAN UP ────────────────────────────────────────────────────────────────────

cap.release()
out.release()
print("Done – saved with overlay to:", output_vid)

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
Adding overlay: 100%|███████████████████| 17204/17204 [00:05<00:00, 3387.12it/s]

Done – saved with overlay to: /Users/johnmarshall/Documents/Analysis/miniscope_analysis/miniscopeLinearTrack/2025.1/m989/2025_01_04/My_WebCam/with_velocity_overlay.mp4


In [ ]:
len(vel_array)

In [ ]:
binned.index

In [ ]:
binned.loc[20563]

In [ ]:
vel_array[20562]

In [ ]:
n_frames